In [1]:
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
#!pip install captum

# Load & prep data

In [2]:
import pickle

In [4]:
# Connect to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

p_file_path = "/content/drive/My Drive/NLP/explanations_llama3_non_it.pkl"
with open(p_file_path, "rb") as file:
  explanations = pickle.load(file)

Mounted at /content/drive


In [6]:
# Iterate through all objects and transform input_tokens
for explanation in explanations:
    explanation.input_tokens = [token.replace('Ġ', '') for token in explanation.input_tokens]
    # explanation.input_tokens = [token.replace('G', '') for token in explanation.input_tokens]
    explanation.input_tokens = [token.replace('Ģ', '') for token in explanation.input_tokens]
    explanation.input_tokens = [token.replace('<|begin_of_text|>', '') for token in explanation.input_tokens]
    #explanation.input_tokens = [token for token in explanation.input_tokens if token != '']


# Verify the transformation
for explanation in explanations:
    print(explanation.input_tokens)

['', '"I', 'went', 'on', 'a', 'message', 'and', 'there', 'was', 'a', 'picture', 'of', 'a', 'man', "'s"]
['', '"Why', 'would', 'a', 'guy', 'with', 'a', 'small', 'penis', 'put', 'his']
['', 'I', 'am', 'certain', 'you', 'agree', 'that', 'LDS', 'people', 'are', 'a', 'bunch', 'of', 'brain', 'w', 'ashed', ',', 'homophobic', ',']
['', 'I', "'ve", 'tried', 'lots', 'of', 'weird', 'looking', 'sex', 'toys', 'in', 'my', 'day', ',', 'but', 'the', 'clit', 'pump', '-', 'which', 'is']
['', 'I', 'âĻll', 'say', 'this', ':', 'Nep', 'hi', 'was', 'a', 'self', '-right', 'eous', 'pain', 'in']
['', 'Mark', 'Parkinson', 'tweeted', 'a', 'photo', 'of', 'a', 'gigantic', 'penis', 'with', 'the', 'words']
['', 'She', 'saw', 'the', 'curve', 'of', 'her', 'hips', 'and', 'the', 'shape']
['', 'The', 'third', 'reads', ':', 'âľ', 'Hi', 'you', 'fucking', 'filthy', 'jew', 's', ',', 'I', 'just', 'wanted']
['', 'âľAs', 'soon', 'as', 'they', 'come', 'off', ',', 'I', 'âĻm', 'gonna', 'beat']
['', 'âľ', 'Plain', 'and', 'simple', '

In [7]:
# Step 1: Extract input and put tokens together
sentences = []
sentences_w_score = []

#for explanation in explanations:
#      sentence = ' '.join(explanation.input_tokens)
#      sentences.append(sentence)

#print(sentences)


In [ ]:
#for index_exp, explanation in enumerate(explanations):
#    for index_attr, attr in enumerate(explanation.seq_attr):
#        if abs(attr) > 3:
#            # To print the attribute score and the corresponding input_token to that score
#            print(f"attr: {attr}, corresponding input_token: {explanation.input_tokens[index_attr]}")
#            # print the explanation
#            print(f"Explanation: {explanation.input_tokens}")
#            # print the index of the explanation
#            print(f"Index of Explanation: {index_exp}")
#            # print the sentence
#            print(f"Sentence: {sentences[index_exp]}")
#            print("\n")

# Create and parse words


In [8]:
import torch

In [ ]:
def tokens_to_words(tokens, seq_attr):
    words = []
    word_importances = []

    current_word = ""
    current_importance = 0.0

    for token, importance in zip(tokens, seq_attr):
        # Remove the special token prefix (assuming BPE tokenization used here)
        token = token.replace('▁', '')

        if current_word == "":
            current_word = token
            current_importance = importance
        else:
            if token.startswith('"') or token.startswith("'") or token.isalpha():
                words.append(current_word)
                word_importances.append(current_importance)

                current_word = token
                current_importance = importance
            else:
                current_word += token
                current_importance += importance

    # Append the last word and its importance
    words.append(current_word)
    word_importances.append(current_importance)

    return words, word_importances

In [ ]:
#words, word_importances = tokens_to_words(explanations[1].input_tokens, explanations[1].seq_attr)

for explanation in explanations:
  words, word_importances = tokens_to_words(explanation.input_tokens, explanation.seq_attr)
  sentence = ' '.join(explanation.input_tokens)
  sentences.append(sentence)
  sentences_w_score.append(word_importances)
  #print(words)
  #print(word_importances)
  #print(sentences)
  #print(sentences_w_score)
  #print("\n")

print(sentences_w_score)


[[tensor(-0.7829, device='cuda:0'), tensor(-2.4494, device='cuda:0'), tensor(-1.4874, device='cuda:0'), tensor(-0.4440, device='cuda:0'), tensor(-0.9387, device='cuda:0'), tensor(-0.1051, device='cuda:0'), tensor(-2.8808, device='cuda:0'), tensor(3.3173, device='cuda:0'), tensor(-1.4834, device='cuda:0'), tensor(0.1630, device='cuda:0'), tensor(1.2710, device='cuda:0'), tensor(-1.3979, device='cuda:0'), tensor(-2.1511, device='cuda:0'), tensor(1.4880, device='cuda:0')], [tensor(3.5942, device='cuda:0'), tensor(10.9009, device='cuda:0'), tensor(2.7347, device='cuda:0'), tensor(3.1478, device='cuda:0'), tensor(3.1245, device='cuda:0'), tensor(3.8307, device='cuda:0'), tensor(6.7620, device='cuda:0'), tensor(7.5551, device='cuda:0'), tensor(7.4122, device='cuda:0'), tensor(-3.4147, device='cuda:0')], [tensor(0.5321, device='cuda:0'), tensor(4.5185, device='cuda:0'), tensor(15.7539, device='cuda:0'), tensor(9.7204, device='cuda:0'), tensor(10.0030, device='cuda:0'), tensor(9.5076, device='

# Normalization and filtering

In [ ]:
# Normalize each sublist
normalized_data = []

for sublist in sentences_w_score:
    # Convert each tensor to its absolute value
    sublist_abs = [word.abs() for word in sublist]

    # Find the minimum and maximum values in the sublist
    sublist_min = min(sublist_abs).item()
    sublist_max = max(sublist_abs).item()

    # Normalize each tensor in the sublist
    normalized_sublist = [(word.item() - sublist_min) / (sublist_max - sublist_min) for word in sublist_abs]

    # Append normalized sublist to the result
    normalized_data.append(normalized_sublist)
    print(normalized_sublist)


# Display or use normalized_data as needed
# print(normalized_data)

[0.21101159302565856, 0.7298167109629028, 0.43035040341639175, 0.105510546779438, 0.2595236907671443, 0.0, 0.8641328744575789, 1.0, 0.42911058383151757, 0.018029636913371762, 0.36298449750492245, 0.40246633842324403, 0.636958499295773, 0.4305214130143054]
[0.10525056990171532, 1.0, 0.0, 0.050590455547666204, 0.047742815501326655, 0.13422026234164205, 0.49317052206734185, 0.5902892484771479, 0.5727942000822154, 0.08327478605329049]
[0.021484696961371504, 0.2647929062967699, 0.9505499929685092, 0.5822945198143686, 0.5995430927942585, 0.5693048882365487, 0.9261996142458936, 0.2472411875862056, 0.0, 0.013720254698293164, 0.2938799952688777, 0.15976990726814008, 0.2653721893827282, 0.1791060414010455, 0.03504243574567796, 1.0]
[0.0, 0.42439212421901973, 0.6083109070012722, 0.3979180165820851, 0.2223186541362985, 0.11185576518329683, 0.331823792880014, 0.6259524293747901, 0.652166407325252, 0.6364403853703064, 0.3299224813534311, 0.6195650631642176, 0.24540161658776066, 0.10421268203166015, 

# Dependency parsing and analysis

In [9]:
import spacy
import csv
import pandas as pd
from collections import Counter

# Load the English language model in spaCy
nlp = spacy.load("en_core_web_sm")

# Function to extract syntactic patterns from a parsed sentence
def extract_syntactic_patterns(doc):
    patterns = []
    for token in doc:
        patterns.append((token.text, token.dep_, token.head.text, token.head.pos_))
    return patterns

# Step 2: Parse each sentence and collect syntactic features
syntactic_data = []
for sentence in sentences:
    doc = nlp(sentence)
    patterns = extract_syntactic_patterns(doc)
    syntactic_data.append({
        "sentence": sentence,
        "patterns": patterns
    })

# Step 3: Identify common syntactic patterns
all_patterns = [pattern for data in syntactic_data for pattern in data["patterns"]]
pattern_counter = Counter(all_patterns)

# Save the syntactic patterns and their frequencies to a CSV file
patterns_df = pd.DataFrame(pattern_counter.items(), columns=["pattern", "frequency"])
patterns_df.sort_values(by="frequency", ascending=False, inplace=True)
patterns_df.to_csv("syntactic_patterns_dependency.csv", index=False)

# Save the detailed syntactic analysis for each sentence to a CSV file
syntactic_details_df = pd.DataFrame(syntactic_data)
syntactic_details_df.to_csv("syntactic_analysis_dependency.csv", index=False)

# Example of further analysis: Identify most common dependency types
common_deps = pattern_counter.most_common(10)
print("Most common dependency types:")
for pattern, freq in common_deps:
    print(f"{pattern[1]} ({pattern[3]} -> {pattern[2]}): {freq} occurrences")


Most common dependency types:


# Constituent parsing

In [ ]:
!pip install nltk
!pip install benepar

In [ ]:
import csv
import pandas as pd
import nltk
from nltk.tree import Tree
import benepar

# Download necessary NLTK data and benepar model
nltk.download('punkt')
benepar.download('benepar_en3')

# Load the benepar parser
parser = benepar.Parser("benepar_en3")

# Step 1: Load the CSV file and extract the tokens from the desired column is already done


# Function to parse a sentence and return the constituency parse tree
def parse_sentence(sentence):
    doc = nltk.word_tokenize(sentence)
    parse_tree = parser.parse(doc)
    return parse_tree

# Step 2: Parse each sentence and collect constituent parse trees
parsed_data = []
for sentence in sentences:
    parse_tree = parse_sentence(sentence)
    parsed_data.append({
        "sentence": sentence,
        "parse_tree": parse_tree
    })

# Step 3: Extract syntactic patterns and analyze constituent parse trees
def extract_constituent_patterns(parse_tree):
    patterns = []
    for subtree in parse_tree.subtrees():
        if subtree.height() > 2:  # Ignore the leaf nodes
            patterns.append((subtree.label(), ' '.join(subtree.leaves())))
    return patterns

all_patterns = []
for data in parsed_data:
    patterns = extract_constituent_patterns(data["parse_tree"])
    all_patterns.extend(patterns)
    data["patterns"] = patterns

# Save the constituent patterns and their frequencies to a CSV file
patterns_df = pd.DataFrame(all_patterns, columns=["constituent", "phrase"])
pattern_counter = patterns_df.groupby("constituent").size().reset_index(name="frequency")
pattern_counter.sort_values(by="frequency", ascending=False, inplace=True)
pattern_counter.to_csv("constituent_patterns.csv", index=False)

# Save the detailed constituent analysis for each sentence to a CSV file
detailed_data = []
for data in parsed_data:
    detailed_data.append({
        "sentence": data["sentence"],
        "parse_tree": str(data["parse_tree"]),
        "patterns": data["patterns"]
    })
detailed_df = pd.DataFrame(detailed_data)
detailed_df.to_csv("constituent_analysis.csv", index=False)

# Example of further analysis: Print the most common constituents
common_constituents = pattern_counter.head(10)
print("Most common constituents:")
print(common_constituents)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/distributions/distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


Most common constituents:
   constituent  frequency
2           NP         48
11          VP         26
5            S         18
3           PP         11
10         TOP         10
0         ADJP          6
6         SBAR          5
1         ADVP          3
7        SBARQ          2
4          PRT          1


# Further analysis

Currently not working and under construction :(